# Load libraries

In [ ]:
suppressPackageStartupMessages({
    library(dplyr)
    library(Seurat)
    library(BayesPrism)
    library(SingleCellExperiment)
})

# Load GTEx data

In [ ]:
get_counts <- function(){
    fn = here::here("inputs/gtex/_m/",
                    "genes_gtex_v8_counts.txt.gz")
    return(data.table::fread(fn))
}
memCOUNTS <- memoise::memoise(get_counts)

get_pheno <- function(){
    fn = here::here("inputs/gtex/_m/gtex_v8_sample_data.tsv")
    return(data.table::fread(fn) |> filter(SMTS == "Lung"))
}
memPHENO <- memoise::memoise(get_pheno)

select_lung_data <- function(){
                                        # Clean data
    counts <- tibble::column_to_rownames(memCOUNTS(), "Name") |>
        select(any_of(memPHENO()$SAMPID))
    genes  <- memCOUNTS() |> select(Name, Description)
    pheno  <- memPHENO() |> filter(SAMPID %in% colnames(counts))
                                        # Filter low expression
    x <- edgeR::DGEList(counts=counts, genes=genes, samples=pheno)
    keep.x <- edgeR::filterByExpr(x)
    x <- x[keep.x, , keep.lib.sizes=FALSE]
                                        # Update rownames
    bulk_counts <- x$counts |> as.data.frame() |>
        tibble::rownames_to_column("Name") |>
        inner_join(genes, by="Name") |>
        distinct(Description, .keep_all=TRUE) |>
        tibble::column_to_rownames("Description") |>
        select(-Name)
    return(bulk_counts)
}
memDF <- memoise::memoise(select_lung_data)

In [ ]:
bk.dat <- t(memDF())
dim(bk.dat)

In [ ]:
head(rownames(bk.dat))

In [ ]:
colnames(bk.dat) |> head()

# Load single-cell reference

## Load data

In [ ]:
load("../_m/scRNA_HLCA_version2.RData", verbose=TRUE)

In [ ]:
sce

## Extract counts

In [ ]:
rownames(sce) <- rowData(sce)[, "feature_name"]
sc.dat        <- assays(sce)$counts |> t()
dim(sc.dat)

In [ ]:
head(rownames(sc.dat))

In [ ]:
colnames(sc.dat) |> head()

## Extract cell type labels

In [ ]:
nrow(sc.dat)

In [ ]:
cell_types <- sce$cell_type
sort(table(cell_types))

# QC of cell types

Examine the pairwise correlation matrix between cell types. This will give us a sense of quality of specific cell types. The low-quality cell types will cluster together. Depending on this level of clustering, we may need to use a different level of granularity or remove some subclusters.

In [ ]:
plot.cor.phi(input=sc.dat,
             input.labels=sce$compartment,
             title="Cell type correlation",
             cexRow=0.5, cexCol=0.5)

In [ ]:
plot.cor.phi(input=sc.dat,
             input.labels=sce$ann_level_2,
             title="Cell type correlation",
             cexRow=0.5, cexCol=0.5)

In [ ]:
plot.cor.phi(input=sc.dat,
             input.labels=sce$ann_level_3,
             title="Cell type correlation",
             cexRow=0.2, cexCol=0.2, 
             margins=c(2,2))

In [ ]:
plot.cor.phi(input=sc.dat,
             input.labels=sce$cell_type,
             title="Cell type correlation",
             cexRow=0.1, cexCol=0.1, 
             margins=c(2,2))

## Collapse cell types

In [ ]:
sc.datX <- sc.dat[,colSums(sc.dat) >= 3]
dim(sc.datX)

In [ ]:
norm.to.one <- function(ref, pseudo.min){
	G <- ncol(ref)
	phi <- ref/rowSums(ref) * (1-pseudo.min*G) + pseudo.min
	#if the minimum value is greater than zero. simply normalize by total depth
	min.value <- apply(ref,1,min)
	which.row <- min.value>0
	if(any(which.row)){
		phi[which.row,] <- ref[which.row,,drop=F]/rowSums(ref[which.row,,drop=F])
	}
	return(phi)
}

collapse <- function(ref, labels){
	stopifnot(nrow(ref) == length(labels))
	non.na.idx <- !is.na(labels)
	if(sum(!non.na.idx)>0) {
        print(paste("Warning: NA found in the cell type/state labels.",
                    "These cells will be excluded!"))
    }
	labels <- labels[non.na.idx]
	ref <- ref[non.na.idx,]
	labels.uniq <- unique(labels)
	ref.collapsed <- do.call(rbind,
							 lapply(labels.uniq,
							 		function(label.i) 
							 			colSums(ref[labels==label.i,,drop=F])
							 		)
							 )
	rownames(ref.collapsed) <- labels.uniq
	return(ref.collapsed)
}

In [ ]:
ref.ct  <- collapse(ref = sc.datX, labels = sce$cell_type)
ref.ct  <- norm.to.one(ref = ref.ct, pseudo.min = 1E-8)
dim(ref.ct)

In [ ]:
ref.ct[1:2, 1:5]

In [ ]:
ref.ct  <- scale(log2(ref.ct),center=T,scale=F)
ref.ct  <- t(ref.ct)
cor.mat <- cor(ref.ct)
dim(cor.mat)

In [ ]:
for(cell in 1:39){
    print(colnames(cor.mat)[cell])
    print(cor.mat[cell,][cor.mat[cell,] > 0.8])
}

In [ ]:
sce$cell_type[sce$cell_type == "Monocyte-derived Mph"] <- "Monocytes"
sce$cell_type <- droplevels(sce$cell_type)
sort(table(sce$cell_type))

In [ ]:
ref.ct  <- collapse(ref = sc.datX, labels = sce$cell_type)
ref.ct  <- norm.to.one(ref = ref.ct, pseudo.min = 1E-8)
dim(ref.ct)

In [ ]:
ref.ct[1:2, 1:5]

In [ ]:
ref.ct  <- scale(log2(ref.ct),center=T,scale=F)
ref.ct  <- t(ref.ct)
cor.mat <- cor(ref.ct)
dim(cor.mat)

In [ ]:
for(cell in 1:38){
    print(colnames(cor.mat)[cell])
    print(cor.mat[cell,][cor.mat[cell,] > 0.75])
}

## Filter outlier genes (if needed)

In [ ]:
## This is better reduction in correlation (< 0.8)
cell_types <- sce$cell_type
plot.cor.phi(input=sc.dat, pdf.prefix="lung.cor",
             input.labels=cell_types,
             title="Cell type correlation",
             cexRow=0.2, cexCol=0.2, 
             margins=c(2,2))

In [ ]:
sc.stat <- plot.scRNA.outlier(
    input=sc.dat, 
    pdf.prefix="lung.sc_stat",
    cell.type.labels=cell_types,
    species="hs", return.raw=TRUE)

In [ ]:
sc.stat <- plot.scRNA.outlier(
    input=sc.dat, 
    cell.type.labels=cell_types,
    species="hs", return.raw=TRUE)

In [ ]:
head(sc.stat)

In [ ]:
bk.stat <- plot.bulk.outlier(
    bulk.input=bk.dat, sc.input=sc.dat,
    cell.type.labels=cell_types,
    species="hs", return.raw=TRUE)

In [ ]:
plot.bulk.outlier(
    bulk.input=bk.dat, sc.input=sc.dat,
    cell.type.labels=cell_types,
    species="hs", return.raw=FALSE,
    pdf.prefix="lung.bk_stats"
)

In [ ]:
head(bk.stat)

## Clean up single-cell reference data

In [ ]:
sc.dat.filtered <- cleanup.genes(
    input=sc.dat,
    input.type="count.matrix",
    species="hs",
    gene.group=c("Rb", "Mrp", "other_Rb", "chrM", "chrX", 
                 "chrY", "MALAT1"),
    exp.cells=5)

In [ ]:
dim(sc.dat.filtered)

## Concordance between bulk and reference

In [ ]:
plot.bulk.vs.sc(sc.input=sc.dat.filtered,
                bulk.input=bk.dat)

In [ ]:
plot.bulk.vs.sc(sc.input=sc.dat.filtered,
                bulk.input=bk.dat,
                pdf.prefix="lung.bk_vs_sc")

## Subset for coding genes

In [ ]:
sc.dat.filtered.pc <- select.gene.type(sc.dat.filtered, gene.type="protein_coding")

# Construct prism object

## Protein coding only

In [ ]:
lungPrism <- new.prism(
    reference=sc.dat.filtered.pc,
    mixture=bk.dat,
    input.type="count.matrix",
    cell.type.labels=cell_types,
    cell.state.labels=cell_types,
    outlier.cut=0.01,
    outlier.fraction=0.1)

## Save object

In [ ]:
save(lungPrism, file="bayesPrism_object.RData")

# Reproducibility information

In [ ]:
print("Reproducibility information:")
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()